<a href="https://colab.research.google.com/github/baradel2548/ARQUEARIA-PICA-EST-GIO-7/blob/main/PRO_C114_projtemp_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Carregue o conjunto de dados do repositório Github
!git clone https://github.com/ByjusBrazil/PRO_1-1_C114_ProductDataset

Cloning into 'PRO_1-1_C114_ProductDataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
# Faça um dataframe usando o módulo "pandas"
import pandas as pd
dataframe = pd.read_excel("/content/PRO_1-1_C114_ProductDataset/updated_product_dataset.xlsx")
print(dataframe.head())


     Emoção                                              Texto
0  Positiva  ótimo pad amo ddr não quero pad de metal funci...
1  Positiva  excelente pad ótimo produto altamente responsi...
2  Positiva  incrível ótimo ddr pad funciona perfeitamente ...
3  Positiva  amo esperar deslizamento não muito feliz dia 1...
4  Positiva  preço de boa qualidade preço de boa qualidade ...


In [ ]:
dataframe.head()

,Emoção,Texto
0,Positiva,ótimo pad amo ddr não quero pad de metal funci...
1,Positiva,excelente pad ótimo produto altamente responsi...
2,Positiva,incrível ótimo ddr pad funciona perfeitamente ...
3,Positiva,amo esperar deslizamento não muito feliz dia 1...
4,Positiva,preço de boa qualidade preço de boa qualidade ...


In [ ]:
dataframe["Emoção"].unique()

array(['Positiva', 'Negativa', 'Neutra'], dtype=object)

In [ ]:
encode_emotions={"Positiva":0,"Neutra":1,"Negativa":2}

In [ ]:
dataframe.replace(encode_emotions,inplace=True)

In [ ]:
dataframe.head()

,Emoção,Texto
0,0,ótimo pad amo ddr não quero pad de metal funci...
1,0,excelente pad ótimo produto altamente responsi...
2,0,incrível ótimo ddr pad funciona perfeitamente ...
3,0,amo esperar deslizamento não muito feliz dia 1...
4,0,preço de boa qualidade preço de boa qualidade ...


In [ ]:
training_sentences=[]
training_labels = []
# Anexe o texto e as emoções na lista usando o método 'loc'
for i in range(len(dataframe)):
  sentence = dataframe.loc[i,"Texto"]
  training_sentences.append(sentence)
  label = dataframe.loc[i,"Emoção"]
  training_labels.append(label)
# Verifique um texto e uma etiqueta aleatórios da lista
training_sentences[10],training_labels[10]

('decepcionado lento tirar fotos', 1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Defina or parâmetros do Tokenizer
vocab_size=10000
embedding_dim = 16
oov_tok = "<OOV>"
training_size = 20000
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
#Crie um dicionário chamado word_index
word_index = tokenizer.word_index
#Verifique a sequência tokenizada
word_index["o"]

78

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
print(training_sequences[0])
print(training_sequences[1])
print(training_sequences[2])

[3, 4, 9, 10, 2, 27, 4, 5, 28, 6, 3, 11]
[29, 4, 3, 30, 31, 32, 2, 33, 34, 12, 35, 36, 37, 7, 13, 38, 7, 39]
[40, 3, 10, 4, 6, 41, 42, 11]


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Defina os parâmetros das pad_sequences
padding_type = 'post'
max_length = 100
trunc_type='post'
training_padded = pad_sequences(training_sequences,maxlen=max_length,
                                padding=padding_type,truncating=trunc_type)

#Verifique a sequência preenchida
training_padded

array([[  3,   4,   9, ...,   0,   0,   0],
       [ 29,   4,   3, ...,   0,   0,   0],
       [ 40,   3,  10, ...,   0,   0,   0],
       ...,
       [ 96,   2,  17, ...,   0,   0,   0],
       [ 22,  23,  99, ...,   0,   0,   0],
       [ 23, 111,  24, ...,   0,   0,   0]], dtype=int32)

In [12]:
import numpy as np
training_padded = np.array(training_padded)
training_labels=np.array(training_labels)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D

model = tf.keras.Sequential([
    Embedding(vocab_size,embedding_dim,input_length = max_length),Dropout(0.2),
    Conv1D(filters=256,kernel_size=3,activation='relu'),
    MaxPooling1D(pool_size=3),
    Conv1D(filters=128,kernel_size=3,activation='relu'),
    MaxPooling1D(pool_size=3),
    LSTM(128),
    Dense(128,activation='relu'),Dropout(0.2),
    Dense(64,activation='relu'),
    Dense(6,activation='relu'),
])
model.compile(loss = 'sparse_categorical_crossentropy',optimizer='adam',
              metrics=['accuracy'])


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 dropout (Dropout)           (None, 100, 16)           0         
                                                                 
 conv1d (Conv1D)             (None, 98, 256)           12544     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 32, 256)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 128)           98432     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 10, 128)          0         
 1D)                                                    

In [16]:
num_epoch = 5
history = model.fit(training_padded,training_labels,epochs=num_epoch,verbose=2)

Epoch 1/5
1/1 - 0s - loss: 5.1300 - accuracy: 0.4000 - 72ms/epoch - 72ms/step
Epoch 2/5
1/1 - 0s - loss: 4.8087 - accuracy: 0.2667 - 64ms/epoch - 64ms/step
Epoch 3/5
1/1 - 0s - loss: 1.5377 - accuracy: 0.3333 - 58ms/epoch - 58ms/step
Epoch 4/5
1/1 - 0s - loss: 1.4996 - accuracy: 0.3333 - 55ms/epoch - 55ms/step
Epoch 5/5
1/1 - 0s - loss: 1.3224 - accuracy: 0.3333 - 58ms/epoch - 58ms/step


In [17]:
model.save('text_emotion.h5')

In [18]:
#use o método predict() para previsão do modelo
sentence = ["Eu estou feliz em encontrar meus amigos. Estamos planejando ir a uma festa.",
            "Eu tive um dia ruim na escola. Eu me machuquei jogando futebol.",
            "Eu tive um dia ruim na escola. Mas pelo menos ganhei no futebol.",
            "Achei uma flor linda na rua. Tirei uma foto legal",
            "Eu estou feliz em encontrar meus amigos. Estamos planejando ir a uma festa.",
            "Eu tive um dia ruim na escola. Eu me machuquei jogando futebol.",
            "Eu tive um dia ruim na escola. Mas pelo menos ganhei no futebol.",
            "Achei uma flor linda na rua. Tirei uma foto legal",]

sequences = tokenizer.texts_to_sequences(sentence)

padded = pad_sequences(sequences, maxlen=max_length,
                       padding=padding_type, truncating=trunc_type)

result=model.predict(padded)

predict_class = np.argmax(result, axis=1)
predict_class
# {"raiva": 0, "medo": 1, "alegria": 2, "amor": 3, "tristeza": 4, "surpresa": 5}

1/1 [==============================] - 1s 561ms/step


array([1, 1, 1, 1, 1, 1, 1, 1])